In [2]:
import urllib3
from bs4 import BeautifulSoup
import certifi
import logging

In [3]:
# ログレベルを DEBUG に変更
formatter = '%(levelname)s : %(asctime)s : %(message)s'
logging.basicConfig(filename='logger.log', level=logging.DEBUG, format=formatter)

# 従来の出力
logging.info('error{}'.format('outputting error'))
logging.info('warning %s %s' % ('was', 'outputted'))
# logging のみの書き方
logging.info('info %s %s', 'test', 'test')

In [4]:
host = "https://www.panamusica.co.jp"

In [5]:
def get_page(page):
#     url = host + "/ja/product/genre/104/r/000/" + page + "/"
    url = page
    print(url)
    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())
    r = http.request('GET', url)
    return r

In [6]:
def get_contents(spec, num):
    return spec.contents[num] if (len(spec) > num) else ''

In [7]:
def get_string(obj):
    return obj.string if obj is not None else ''

In [8]:
def output(r):
    soup = BeautifulSoup(r.data, 'html.parser')
    contents = soup.find("ol")

    song_list = contents.find_all("li")
    return_list = []

    for song in song_list:
        
        title = song.h4.string
        spec = song.find("div", class_="spec")
        
        composer = get_contents(spec, 3)
        publisher = get_contents(spec, 5)
        price = get_string(song.strong)
        path = host + song.a.get("href")

    #     print("曲名：" + title)
    #     print("出版社：" + publisher)
    #     print("作編曲者：" + composer)
    #     print("単価（税込）：" + price)
    #     print("リンク：" + path)

        # データをリストに保持
        list = []
        list.append(title)
        list.append(composer)
        list.append(publisher)
        list.append(price)
        list.append(path)
        
        return_list.append('$'.join(list).replace(u'\xa0', u'').replace(u'\u3000', u''))
        print('$'.join(list))
        
    return return_list

In [7]:
for i in range(10):
    try:
        page = i * 10
        r = get_page(str(page))
        output_list = output(r)
        
        path_w = 'list.txt'
        with open(path_w, mode='a') as f:
            f.write('\n'.join(output_list))
            f.write('\n')
            f.close()
        
        logging.info('Complete!: %d', i)
    except Exception as inst:
        logging.error(inst)

https://www.panamusica.co.jp/ja/product/genre/104/r/000/0/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/10/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/20/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/30/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/40/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/50/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/60/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/70/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/80/
https://www.panamusica.co.jp/ja/product/genre/104/r/000/90/


In [9]:
url_list = [
"https://www.panamusica.co.jp/ja/product/genre/1/r/000/1090",
"https://www.panamusica.co.jp/ja/product/genre/1/r/000/1630",
"https://www.panamusica.co.jp/ja/product/genre/101/r/000/260",
"https://www.panamusica.co.jp/ja/product/genre/2/r/000/40",
"https://www.panamusica.co.jp/ja/product/genre/2/r/000/580",
"https://www.panamusica.co.jp/ja/product/genre/2/r/000/690",
"https://www.panamusica.co.jp/ja/product/genre/2/r/000/1080",
"https://www.panamusica.co.jp/ja/product/genre/2/r/000/1390",
"https://www.panamusica.co.jp/ja/product/genre/2/r/000/1630",
"https://www.panamusica.co.jp/ja/product/genre/2/r/000/1820",
"https://www.panamusica.co.jp/ja/product/genre/2/r/000/1850"

# "https://www.panamusica.co.jp/ja/product/genre/2/r/000/320",
# "https://www.panamusica.co.jp/ja/product/genre/1/r/000/230",
# "https://www.panamusica.co.jp/ja/product/genre/1/r/000/240",
# "https://www.panamusica.co.jp/ja/product/genre/1/r/000/620",
# "https://www.panamusica.co.jp/ja/product/genre/1/r/000/780"
]

for url in url_list:
    # try:
    r = get_page(url)
    output_list = output(r)

    path_w = 'list.txt'
    with open(path_w, mode='a') as f:
        f.write('\n'.join(output_list))
        f.write('\n')
        f.close()

#     logging.info('Complete!: %d', i)
    # except Exception as inst:
    #     logging.error(inst)

https://www.panamusica.co.jp/ja/product/genre/1/r/000/1090
¤á¤¶¤»¡ª¹ç¾§¥³¥ó¥¯¡¼¥ë<¥Ò¥Ã¥È¡¦¥½¥ó¥°ÊÔ>$ÆüÌ¾»ÒµªÂå¡¦ÀÄ»³¤·¤ª¤ê $¥É¥ì¥ß³ÚÉè½ÐÈÇ¼Ò$1,728±ß$https://www.panamusica.co.jp/ja/product/13073/
º®À¼¹ç¾§¤Î¤¿¤á¤ÎµÜºê½Ù¥¢¥Ë¥áÌ¾¶Ê½¸¡Ö¤µ¤¯¤é¤ó¤Ü¤Î¼Â¤ëº¢¡×$»ûÅè¡¡Î¦Ìé $¥«¥ï¥¤½ÐÈÇ$2,052±ß$https://www.panamusica.co.jp/ja/product/13068/
ÌµÈ¼ÁÕº®À¼¹ç¾§ÁÈ¶Ê­·¡Ö¥Ô¥å¥·¥¹¡×¡Ê¼õÃíÀ¸»º¡Ë$»³ËÜ¡¡½ã¥Î²ð $²»³ÚÇ·Í§¼Ò$2,171±ß$https://www.panamusica.co.jp/ja/product/13055/
ÌµÈ¼ÁÕº®À¼¹ç¾§ÁÈ¶Ê­¶¡Ö¥Ñ¥ó¥¿¡¦¥ì¥¤¡×$»³ËÜ¡¡½ã¥Î²ð $²»³ÚÇ·Í§¼Ò$2,052±ß$https://www.panamusica.co.jp/ja/product/13054/
º®À¼¹ç¾§¤È¥Ô¥¢¥Î¤Î¤¿¤á¤Î¡Ö¸Å»öµ­ðó²Î(¤Û¤á¤¦¤¿)¡×$ÎëÌÚ¡¡µ±¾¼ $²»³ÚÇ·Í§¼Ò$2,700±ß$https://www.panamusica.co.jp/ja/product/13047/
º®À¼¹ç¾§¤È¥Ô¥¢¥Î¤Î¤¿¤á¤Ë¡Öµ¨Àá¤¬ËÍ¤òÏ¢¤ìµî¤Ã¤¿¤¢¤È¤Ë¡×¡Ê¼õÃíÀ¸»º¡Ë$¼ãÎÓ¡¡Àé½Õ $²»³ÚÇ·Í§¼Ò$2,354±ß$https://www.panamusica.co.jp/ja/product/12986/
º®À¼¹ç¾§ÁÈ¶Ê¡Ö¤ê¤³¤ë¤É¡×$»³ËÜ¡¡½ã¥Î²ð $²»³ÚÇ·Í§¼Ò$1,944±ß$https://www.panamusica.co.jp/ja/product/12984/
º®À¼¹ç¾§ÁÈ¶Ê¡Ö¤½¤ì¤ÏÂÔ¤Ã¤Æ¤¤¤ë¡ª¡×$Âç·§¡¡¿ò»Ò $²»³

UnicodeEncodeError: 'cp932' codec can't encode character '\xa4' in position 0: illegal multibyte sequence